In [1]:
from sklearn import tree
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Read the csv file into a pandas DataFrame

data = pd.read_csv('Well1.csv')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data = data.fillna(0)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   DATEPRD                   746 non-null    object 
 1   WELL_BORE_CODE            746 non-null    object 
 2   AVG_DOWNHOLE_PRESSURE     746 non-null    float64
 3   AVG_DOWNHOLE_TEMPERATURE  746 non-null    float64
 4   AVG_DP_TUBING             746 non-null    float64
 5   AVG_CHOKE_SIZE_P          746 non-null    float64
 6   AVG_WHP_P                 746 non-null    float64
 7   AVG_WHT_P                 746 non-null    float64
 8   DP_CHOKE_SIZE             746 non-null    float64
 9   BORE_OIL_VOL              746 non-null    float64
 10  BORE_GAS_VOL              746 non-null    float64
 11  BORE_WAT_VOL              746 non-null    float64
dtypes: float64(10), object(2)
memory usage: 70.1+ KB


In [3]:
#Getting the rows with non null values, 
data1= data.loc[(data!=0).any(axis=1)]
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 746 entries, 0 to 745
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   DATEPRD                   746 non-null    object 
 1   WELL_BORE_CODE            746 non-null    object 
 2   AVG_DOWNHOLE_PRESSURE     746 non-null    float64
 3   AVG_DOWNHOLE_TEMPERATURE  746 non-null    float64
 4   AVG_DP_TUBING             746 non-null    float64
 5   AVG_CHOKE_SIZE_P          746 non-null    float64
 6   AVG_WHP_P                 746 non-null    float64
 7   AVG_WHT_P                 746 non-null    float64
 8   DP_CHOKE_SIZE             746 non-null    float64
 9   BORE_OIL_VOL              746 non-null    float64
 10  BORE_GAS_VOL              746 non-null    float64
 11  BORE_WAT_VOL              746 non-null    float64
dtypes: float64(10), object(2)
memory usage: 75.8+ KB


In [61]:
data.loc[data['AVG_DOWNHOLE_TEMPERATURE'] == 0]

,DATEPRD,WELL_BORE_CODE,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL,GOR,DEL_P


In [4]:
data = data.loc[data['BORE_OIL_VOL'] > 0]
data = data.loc[data['AVG_CHOKE_SIZE_P'] > 40]
data['GOR']= data['BORE_GAS_VOL']/data['BORE_OIL_VOL']
data['DEL_P']= data['AVG_DOWNHOLE_PRESSURE']-data['AVG_WHP_P']
data = data.loc[data['GOR'] > 0]

data.head()


,DATEPRD,WELL_BORE_CODE,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL,GOR,DEL_P
15,22-Apr-14,NO 15/9-F-1 C,289.421,106.353,182.059,43.34345,107.362,37.939,78.935,631.0,90439.0,0.0,143.326466,182.059
16,23-Apr-14,NO 15/9-F-1 C,270.240,107.644,171.053,47.16752,99.187,60.757,70.627,1166.0,165720.0,0.0,142.126930,171.053
17,24-Apr-14,NO 15/9-F-1 C,262.843,107.869,168.242,47.73231,94.601,63.047,66.049,1550.0,221707.0,0.0,143.036774,168.242
18,25-Apr-14,NO 15/9-F-1 C,255.527,107.971,165.539,48.53377,89.988,64.547,61.405,1249.0,178064.0,0.0,142.565252,165.539
19,26-Apr-14,NO 15/9-F-1 C,247.199,108.052,162.422,49.84450,84.777,65.724,56.148,1346.0,192602.0,0.0,143.092125,162.422


In [5]:
df = data[['AVG_DOWNHOLE_PRESSURE','AVG_CHOKE_SIZE_P','AVG_WHP_P','DP_CHOKE_SIZE','GOR','DEL_P']]
#feature_names = data.columns
#Question - Can we make IID assumption?
feature_names = df.columns
df.head()

,AVG_DOWNHOLE_PRESSURE,AVG_CHOKE_SIZE_P,AVG_WHP_P,DP_CHOKE_SIZE,GOR,DEL_P
15,289.421,43.34345,107.362,78.935,143.326466,182.059
16,270.240,47.16752,99.187,70.627,142.126930,171.053
17,262.843,47.73231,94.601,66.049,143.036774,168.242
18,255.527,48.53377,89.988,61.405,142.565252,165.539
19,247.199,49.84450,84.777,56.148,143.092125,162.422


In [16]:
feature_names

Index(['AVG_DOWNHOLE_PRESSURE', 'AVG_CHOKE_SIZE_P', 'AVG_WHP_P',
       'DP_CHOKE_SIZE', 'GOR', 'DEL_P'],
      dtype='object')

In [6]:
X = df
y = data['BORE_OIL_VOL'].values.reshape(-1,1)

In [7]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
X_train.shape

(303, 6)

In [9]:
from sklearn.preprocessing import StandardScaler

x_scaler = StandardScaler().fit(X_train)
X_train_s = x_scaler.transform(X_train)
X_test_s = x_scaler.transform(X_test)

In [10]:
from sklearn.preprocessing import StandardScaler

y_scaler = StandardScaler().fit( y_train)
y_train_s = y_scaler.transform( y_train)
y_test_s = y_scaler.transform(y_test)

In [11]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_s, y_train_s)
score = model.score(X_test_s, y_test_s)
print(f"R2 Score: {score}")


R2 Score: 0.8404502654052934


In [12]:
from sklearn.metrics import mean_squared_error, r2_score
predicted_train = model.predict(X_test_s)
# YOUR CODE HERE
# Score the prediction with mse and r2
mse = mean_squared_error(y_test_s, predicted_train)
r2 = r2_score(y_test_s, predicted_train)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

Mean Squared Error (MSE): 0.18284721962035216
R-squared (R2 ): 0.8404502654052934


In [13]:
# Call the `score` method on the model to show the r2 score

# YOUR CODE HERE
model.score(X_test_s, y_test_s)

0.8404502654052934

In [14]:
importances = model.coef_
importances

array([[-0.21227707,  0.39937129,  0.13339367,  0.6902113 , -0.14304404,
        -0.25701307]])

In [20]:
sorted(zip(model.coef_,feature_names), reverse=True)

[(array([ 7.69731073e+12,  1.67425538e+01, -7.69731073e+12,  1.01766982e+01,
         -2.92635719e+00, -7.69731073e+12]),
  'AVG_DOWNHOLE_PRESSURE')]

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier (n_estimators=200)
rf = rf.fit(X_train_s,y_train)
model.fit(X, y)
score = model.score(X, y)
print(f"R2 Score: {score}")

C:\Users\RAKHI\Anaconda3\envs\mlproject\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


R2 Score: 0.8419008050034743


In [19]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.17229014826817596, 'GOR'),
 (0.17020923715403175, 'AVG_CHOKE_SIZE_P'),
 (0.16756933209748823, 'DEL_P'),
 (0.16637079446506708, 'AVG_DOWNHOLE_PRESSURE'),
 (0.1626452756734866, 'DP_CHOKE_SIZE'),
 (0.16091521234175044, 'AVG_WHP_P')]